In [17]:
import pandas as pd
import requests
import datetime as DT

# Создаем запрос (можно не трогать границу, файл всегда содержит всю историческую инфу)
url = 'https://translom.ru/local/tools/graph/getindex.php?date_from=13.01.2023&date_to=20.01.2023&basis=217%2C217%2C229'
request = requests.get(url)

# Полученый объект конвертируем в json файл
raw_df = request.json()

# Из json по ключам и позиции вытаскиваем вложенные списки
index_df = raw_df['full_period'][1]['data']

# Создаем пустой список для наполнения интересующими данными
index_list = []

# Циклом добавляем данные в пустой список
for i in index_df:
    index_list.append(i[1])

# Создаем функции для генерации даты с указаными границами и шагом (спасибо StackOverflow)
# Функция 1


def generate_range_dates(start_date, end_date, day_step) -> list:
    date_1 = min(start_date, end_date)
    date_2 = max(start_date, end_date)

    items = [date_1]

    while date_1 < date_2:
        date_1 += DT.timedelta(days=day_step)
        items.append(date_1)

    return items

# Функция 2


def d2s(date):
    return date.strftime("%d/%m/%Y")


# Создаем параллельный список с датами
start_date = DT.date(2015, 12, 31)  # Можно оттюнить
end_date = DT.date(2023, 1, 20)  # И это тоже
date_list = []

# Этап генерации списка дат
items = generate_range_dates(
    start_date=start_date, end_date=end_date, day_step=7)

for date in items:
    date_list.append(date)

# Преобразуем в формат даты
date_list = pd.to_datetime(date_list)

# Создаем датафрейм из 2-х списков (дата и целевой показатель)
data = {'date': date_list, 'value': index_list}  # Сначала словарь из списков
final_df = pd.DataFrame(data)  # Затем сам df

# Пишем в файл (путь оттюнить надо при необходимости)
with pd.ExcelWriter('/Users/hotdonkey/Documents/GitHub/data_science_learning/code/pet_project/data/translom_copper.xlsx') as writer:
    final_df.to_excel(writer, sheet_name='dynamic')
